WINDOW SPLIT

In [1]:
from src.execution import WindowSplittingExecution
from numpy import random
import os

2025-04-28 09:38:23.696086: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 09:38:23.761546: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 09:38:23.825683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745825903.884903   10217 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745825903.904719   10217 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 09:38:24.034985: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Coral Split
========================

In [ ]:
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
#TEST_SIZE: float = 0.4
BATCH_SPLIT: bool = True
BATCHSIZE: int = 1800 # real-data: 1800, synthetic-data: 326
#SEED: int = 6378 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "." # Decimal separator of the csv file

folder = sorted([f.path for f in os.scandir("assets/data/coral-used") if f.is_file()])

#used: test_size in [0.95, 0.75, 0.5, 0.25]

for file in folder:
    for test_size in [0.07]:
        WindowSplittingExecution.execute(file, 
                                             BATCH_SPLIT, 
                                             VALIDATION_SPLIT, 
                                             test_size, 
                                             236, 
                                             BATCHSIZE, 
                                             INTERPOLATION, 
                                             WINDOWSIZE, 
                                             SEP, 
                                             DECIMAL)

In [2]:
DATA: str = "assets/data/real-data.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
#TEST_SIZE: float = 0.4
BATCH_SPLIT: bool = True
BATCHSIZE: int = 1800 # real-data: 1800, synthetic-data: 326
#SEED: int = 6378 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "," # Decimal separator of the csv file

for test_size in [0.95, 0.9, 0.85, 0.8, 0.5]:
    seeds: list[int] = random.randint(0, 32000, 10)
    for seed in seeds:
        WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 test_size, 
                                 seed, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)

Anzahl an Bleche in der Gesamtmatrix 142
142.0
Richtige Blechnummern umgerechnet der Validierungsdaten [18, 21, 32, 102, 130, 28, 98, 167, 26, 16, 80, 73, 79, 99, 165, 156, 126, 120, 66, 15, 68, 137, 152, 114, 76, 143, 44, 37, 20, 47, 83, 27, 151, 163, 39, 49, 56, 53, 40, 162, 69, 122, 71, 142, 148, 117, 36, 159, 38, 145, 57, 153, 33, 25, 161, 164, 72, 50, 112, 128, 105, 78, 109, 116, 34, 110, 17]
Richtige Blechnummern umgerechnet der Testdaten [113, 134, 58, 103, 77, 104, 135, 146, 131, 74, 169, 55, 141, 144, 168, 106, 118, 51, 108, 101, 46, 42, 31, 41, 107, 65, 155, 70, 111, 61, 84, 52, 133, 136, 48, 85, 75, 115, 35, 81, 14, 63, 29, 82, 60, 149, 59, 140, 24, 23, 166, 147, 150, 62, 132, 158, 54, 22, 125, 138, 19, 154, 64, 67, 123, 160, 124, 30]
Blechnummern für Validierungsdaten, abgezählt vom Array nicht die Originaldaten [5, 8, 19, 77, 104, 15, 73, 139, 13, 3, 67, 60, 66, 74, 137, 129, 100, 95, 53, 2, 55, 111, 125, 89, 63, 116, 31, 24, 7, 34, 70, 14, 124, 135, 26, 36, 43, 40, 27, 13

TUNE MODEL

In [ ]:
from src.execution import CNNTuningExecution

In [ ]:
import os

folder = sorted([f.path for f in os.scandir("build/window_split/tune_test") if f.is_dir()])

for folder in folder:
    CNNTuningExecution.execute(folder)


In [ ]:
from src.experiment import MultipleCNNTrainingExperiment
from src.execution import CNNTrainingExecution
import json
import os

data_folders = sorted([f.path for f in os.scandir("build/tune/tune_test") if f.is_dir()])

for folder in data_folders:
    best_hp = json.load(open(f"{folder}/best-hyperparameters.json", "r"))
    meta= json.load(open(f"{folder}/metadata.json", "r"))

    CNNTrainingExecution.execute(f"{folder}/best-model.h5", meta["data"], "cnn.h5", best_hp["learning_rate"])

TRAIN MODEL

In [ ]:
from src.execution import CNNTrainingExecution
import os
import json

In [ ]:
DATA_FOLDER = "build/window_split/real-data/1744034830"
SAVE_FILENMAME = "cnn.h5" # With file extension

data_folders = sorted([f.path for f in os.scandir("build/tune/tune_test") if f.is_dir()])

for folder in data_folders:
    best_hp = json.load(open(f"{folder}/best-hyperparameters.json", "r"))

    CNNTrainingExecution.execute(f"{folder}/best-model.h5", DATA_FOLDER, "cnn.h5", best_hp["learning_rate"])

VALIDATE MODEL

In [ ]:
from src.execution import CNNValidationExecution 

In [ ]:
MODEL_FILE = "build/soft-start/cnn/1743863962"
DATA_FOLDER = "build/window_split/real-data/1743522733"

CNNValidationExecution.execute(MODEL_FILE, DATA_FOLDER)

In [ ]:
from src.execution import CNNSoftStartExecution

In [ ]:
MODEL_FILE = "build/train/cnn-synthetic-data-offset/1742244525/cnn-synthetic-offset.h5"
DATA_FOLDER = "build/window_split/real-data/1742552872"
SAVE_FILENAME = "cnn-retrained.h5"
LEARNING_RATE = 0.000029335291375872573

CNNSoftStartExecution.execute(MODEL_FILE, DATA_FOLDER, SAVE_FILENAME, LEARNING_RATE)

In [ ]:
from src.experiment import MultipleCNNTrainingExperiment

In [ ]:
MultipleCNNTrainingExperiment.start(N=3, 
                                    test_size=0.8, 
                                    model_file="assets/models/untrained/sim_data_preprocessed_offset_multiplitiv/1743667824/best-model.h5", 
                                    data_file="assets/data/upsampled/sim_data_preprocessed_upsampled.csv",
                                    learning_rate=0.0005110140569138342, 
                                    generate_new_split=True,
                                    sep=";",
                                    decimal=".", 
                                    batchsize=1800, 
                                    batch_split=True, 
                                    interpolation=False)

In [ ]:
from src.experiment import SoftStartExperiment

In [ ]:
import os 

DATA_FOLDER = "build/window_split/soft-start"
MODEL_FILE = "assets/models/untrained/sim_data_preprocessed/1743665223/best-model.h5"

data_folders = sorted([f.path for f in os.scandir(DATA_FOLDER) if f.is_dir()])

for data_folder in data_folders:
    SoftStartExperiment.run(model_file=MODEL_FILE, 
                            data_folder=data_folder,
                            lr_factors=[0.1],
                            unfreezed_layers=[12])

In [ ]:
from src.experiment import MultipleCNNValidationExperiment

VALIDATEEEEE

In [ ]:
import os
from src.experiment import MultipleCNNValidationExperiment
N = 8
MAKE_VALIDATION = True

MultipleCNNValidationExperiment.start("build/train/diff-models-trained-on-same-80", N, MAKE_VALIDATION, None)

In [ ]:
from src.execution import CNNValidationExecution

In [ ]:
import os

DATA_FOLDER = "build/window_split/real-data-20"
N = 10

for data_folder in DATA_FOLDER:
    folder = sorted([f.path for f in os.scandir(DATA_FOLDER) if f.is_dir()])

for data_folder in folder[-N:]:
    CNNValidationExecution.execute(model_folder="", 
                                   data_folder=data_folder)


In [ ]:
import os
import json
import pandas as pd
from pandas import DataFrame


FOLDER = "build/validate/real-20-real"
folder = sorted([f.path for f in os.scandir(FOLDER) if f.is_dir()])

df = DataFrame(columns=["Verstellweg_Phi_loss", 
                        "Verstellweg_Phi_mean_absolute_error", 
                        "Verstellweg_X_loss", 
                        "Verstellweg_X_mean_absolute_error",
                        "Verstellweg_Y_loss",
                        "Verstellweg_Y_mean_absolute_error",
                        "loss"])

for folder in folder:
    with open(os.path.join(folder, "validation_results.json"), "r") as file:
        d = json.load(file)
        results = d["results"]
        df = pd.concat([df, pd.DataFrame([results])], ignore_index=True)

# Calculate mean and variance for each column
mean_values = df.mean()
variance_values = df.std()

print("Mean values:")
print(mean_values)

print("\nStd values:")
print(variance_values)


In [ ]:
from src.show import plot_mean_with_uncertainty

plot_mean_with_uncertainty("build/validate_average/plot-this")

In [ ]:
import numpy as np

array = np.load('build/window_split/experiment-data-subset_used-1/1744619892/x-train-scaled.npy')
print(len(array))